In [24]:
import pandas as pd
import numpy as np
l = 0.1

def f(x, beta):
    return beta[0] * x ** (beta[1]) * np.e ** (beta[2] * x)

def r(y, x, beta, fun):
    res = y - fun(x, beta)
    return res

def r_adj(y, x, beta, fun):
    res = r(y, x, beta, fun)
    if res < 0:
        res *= 0.9
    return res

def drdb(x, beta, j):
    if j == 0:
        return -x ** (beta[1]) * np.e ** (beta[2] * x)
    elif j == 1:
        return -beta[0] * np.log(x) * (x ** (beta[1])) * (np.e ** (beta[2]* x))
    elif j == 2:
        return -beta[0] * (x ** (beta[1]+ 1)) * (np.e ** (beta[2]* x))
    else:
        print("error")

def cost(ys, xs, beta, fun):
    return sum([r_adj(ys[i], x, beta, fun) ** 2 for i, x in enumerate(xs)])

def costd(y, x, beta, fun, funp):
    grad = funp(x, beta)
    return np.array([r_adj(y, x, beta, fun) * grad[j] for j in range(3)])

In [2]:
set1 = [150, 150.909090909091, 151.818181818182, 152.727272727273, 151.136363636364, 152.045454545455, 152.954545454545, 153.863636363636, 152.272727272727, 153.181818181818, 154.090909090909, 152.5, 153.409090909091, 154.318181818182, 155.227272727273, 146.136363636364, 153.276074700493, 154.013331445122, 158.560917921974, 159.292249954471, 159.030552209196, 159.761357745856, 160.491343993796, 161.031444520339, 161.760006904183, 162.755232002254, 157.695704865228, 158.688765798209, 158.794474873158, 155.71532630068, 156.783635334823, 157.853036709491, 158.762127618582, 159.671218527673]
set2 = [162.5, 163.484848484849, 164.469696969697, 170.454545454546, 171.439393939394, 177.424242424242, 178.409090909091, 181.89393939394, 182.878787878788, 183.863636363637, 182.348484848485, 183.333333333333, 184.318181818182, 180.30303030303, 181.287878787879, 182.272727272727, 185.027213886437, 187.646775242667, 188.421257387558, 189.085611707775, 189.858480807301, 190.630483461648, 191.5915700363, 192.361629765503, 193.130826199045, 194.218513249495, 195.306779473028, 196.395625178043, 197.691713257043, 198.878700508003, 195.004975834965, 196.189215910647, 197.174064395495, 198.158912880344]
data = set2

In [3]:
xs = [i + 1 for i in range(len(data))]
num = range(len(xs))

In [4]:
for i in range(-5, 5):
    print(i)

-5
-4
-3
-2
-1
0
1
2
3
4


In [5]:
beta = [162.5, 0.03, 0.004]
print(cost(ys, xs, beta))

NameError: name 'ys' is not defined

In [6]:
search = []
for i in range(4):
    for j in range(4):
        for k in range(4):
            search.append("{} {} {}".format(i, j, k))
search = set(search)
search = [np.array([int(c) for c in s.split(" ")]) for s in search]
search

[array([1, 0, 0]),
 array([2, 1, 2]),
 array([3, 2, 3]),
 array([1, 2, 3]),
 array([0, 2, 0]),
 array([0, 1, 2]),
 array([1, 0, 1]),
 array([3, 0, 2]),
 array([3, 0, 1]),
 array([2, 2, 2]),
 array([2, 0, 1]),
 array([0, 3, 2]),
 array([1, 0, 2]),
 array([2, 1, 0]),
 array([3, 2, 1]),
 array([2, 3, 2]),
 array([3, 3, 2]),
 array([2, 3, 1]),
 array([0, 0, 2]),
 array([1, 1, 3]),
 array([2, 0, 3]),
 array([2, 2, 3]),
 array([2, 3, 3]),
 array([1, 3, 1]),
 array([3, 2, 2]),
 array([3, 1, 2]),
 array([1, 1, 1]),
 array([1, 3, 3]),
 array([1, 2, 2]),
 array([1, 2, 1]),
 array([3, 0, 0]),
 array([0, 1, 3]),
 array([3, 1, 0]),
 array([0, 0, 3]),
 array([0, 0, 1]),
 array([1, 1, 0]),
 array([0, 2, 1]),
 array([2, 2, 0]),
 array([0, 2, 2]),
 array([0, 3, 3]),
 array([1, 3, 0]),
 array([0, 3, 0]),
 array([3, 3, 0]),
 array([3, 2, 0]),
 array([1, 0, 3]),
 array([2, 1, 3]),
 array([2, 1, 1]),
 array([3, 3, 3]),
 array([1, 1, 2]),
 array([3, 1, 1]),
 array([1, 2, 0]),
 array([0, 0, 0]),
 array([3, 3

In [7]:
def model_growth(vals, days, max_iter=50):
    beta = np.array([vals[0], np.log(vals[-1] / vals[0]) / np.log(days[-1] - days[0]) * 4 / 5, 0.002])

    for n in range(max_iter):
        jacobian = np.zeros((len(days), 3))
        for i in range(len(days)):
            row = costd(vals[i], days[i], beta)
            if i == len(days) - 1:
                row = row / 100
            for j in range(3):
                jacobian[i][j] = row[j]
        residuals = np.array([r_adj(vals[i], x, beta) for i, x in enumerate(days)])
        vec = np.dot(np.linalg.inv(np.dot(np.transpose(jacobian), jacobian)), np.transpose(jacobian))
        shift = np.dot(vec, residuals)
        betas = []
        for s in search:
            betas.append([cost(vals, days, beta - shift / 10.0 ** s), beta - shift / 10.0 ** s])
            betas.append([cost(vals, days, beta + shift / 10.0 ** s), beta + shift / 10.0 ** s])
        og_cost = cost(vals, days, beta)
        min_cost = og_cost
        for b in betas:
            if b[0] < min_cost:
                min_cost = b[0]
                beta = b[1]
        if og_cost == min_cost:
            break
    return min_cost, beta, f(days[-1], beta)

In [8]:
for i in range(3, len(data)):
    print(model_growth(data[:i], xs[:i])[2])

164.4801195830456
168.91148430756232
170.68441762829434
175.39626167612292
176.72253077851573
179.80967291536544
181.33005344184684
183.00237950650217
183.99533030208087
184.9016737016234
185.78763642205976
185.0679393773801
185.93014701476866
186.2168970120054
186.59787809214637
187.39633949663846
188.1527012893385
188.89438133497802
189.64938955018422
190.33923377975543
191.08844988283232
191.87651889173048
192.56735271251887
193.38416241123173
194.18227494393025
195.03392725940145
195.99170674753844
196.96307545417352
197.3669416134656
197.75567656242197
198.26266504626912


In [9]:
model_growth(data, xs, 1000)

(141.00791342239765,
 array([1.60151846e+02, 4.49718878e-02, 1.67824289e-03]),
 198.69475454500153)

In [10]:
for x in xs:
    print(-r(0, x, beta))

163.15130173506807
167.24710964028822
169.97245090698206
172.1328965109574
173.98361691884995
175.6390059073067
177.16035913671624
178.58438739503322
179.934831675762
181.227936117547
182.4753070356215
183.68552667127133
184.86512081437888
186.01916790536873
187.15169810432428
188.26596366632597
189.36462742522534
190.44989745833172
191.52362538057554
192.58737945345857
193.64249987561786
194.69014122422942
195.73130547123947
196.7668679790801
197.7975981939076
198.82417628309983
199.84720663465106
200.86722890268567
201.88472711537884
202.90013723914888
203.9138535026512
204.92623371670146
205.93760377542515
206.94826148523856


In [42]:
def f1(x, b):
    return  b[0] * np.e ** (b[1] / x + b[2] * np.log(x))

def f1p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] / x + b[2] * np.log(x))
    grad[1] = (b[0] * np.e ** (b[1] / x + b[2] * np.log(x))) / x
    grad[2] = np.log(x) * b[0] * np.e ** (b[1] / x + b[2] * np.log(x))
    return grad

def initf1b(w=100):
    return [137.4805 * w / 162.5, 0.1675, 0.1216]

def f2(x, b):
    return  x / (b[0] * x + b[1] * x ** (1 / 2) - b[2])

def f2p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] =  - ((x / (b[0] * x + b[1] * x ** (1 / 2) - b[2])) ** 2)
    grad[1] =  - x ** (3 / 2) / ((b[0] * x + b[1] * x ** (1 / 2) - b[2])) ** 2
    grad[2] =  x / (b[0] * x + b[1] * x ** (1 / 2) - b[2]) ** 2
    return grad

def initf2b(w=100):
    return [0.1492, 0.00495, -0.0296]

def f3(x, b):
    return  x ** b[0] * np.e ** (b[1] * x ** b[2])

def f3p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x) * x ** b[0] * np.e ** (b[1] * x ** b[2])
    grad[1] = x ** (b[0] + b[2]) * np.e ** (b[1] * x ** b[2])
    grad[2] = np.log(x) * x * b[1] ** (b[0] + b[2]) * np.e ** (b[1] * x ** b[2])
    return grad

def initf3b(w=100):
    w = 162.5 / w
    return [0.565, 5.08, -0.113]

def f4(x, b):
    return  b[0] * np.e ** (b[1] * x ** 2 + b[2] * x)

def f4p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x ** 2 + b[2] * x)
    grad[1] = x ** 2 * b[0] * np.e ** (b[1] * x ** 2 + b[2] * x)
    grad[2] = x * b[0] * np.e ** (b[1] * x ** 2 + b[2] * x)
    return grad

def initf4b(w=100):
    return [157.38 * w / 162.5, -0.0005342, 0.0215]
######################################
def f5(x, b):
    return  b[0] * x ** 2 + b[1] * x + b[2]

def f5p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** 2
    grad[1] = x
    grad[2] = 1
    return grad

def initf5b(w=100):
    return [-0.066, 3.426, w]
#######################################
def f6(x, b):
    return  b[0] * np.log(x + b[1]) - b[2]

def f6p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x + b[1])
    grad[1] = b[0] / (x + b[1])
    grad[2] = -1
    return grad

def initf6b(w=100):
    return [74.5 * w / 162.5, 22.29, 73.6 * w / 162.5]
########################################
def f7(x, b):
    return  b[0] * np.log(x + b[1])

def f7p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x + b[1])
    grad[1] = b[0] / (x + b[1])
    grad[2] = 0
    return grad

def initf7b(w=100):
    return [56.9 * 162.5, 15.85, 1]
#######################################
def f8(x, b):
    return  (b[0] * x + b[1]) ** (1 / 2)

def f8p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x * (b[0] * x + b[1]) ** (-1 / 2) / 2
    grad[1] = (b[0] * x + b[1]) ** (-1 / 2) / 2
    grad[2] = 0
    return grad

def initf8b(w=100):
    return [937 * w / 162.5, 25069 * w / 162.5, 0]
#######################################
def f9(x, b):
    return  np.log(b[0] * np.e ** (b[1] * x) + 1) + b[2]

def f9p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x) / np.log(b[0] * np.e ** (b[1] * x) + 1)
    grad[1] = b[0] * x * e ** (b[1] * x) / np.log(b[0] * np.e ** (b[1] * x) + 1)
    grad[2] = 1
    return grad

def initf9b(w=100):
    return [0.0366, 2.718, w]
#######################################
def f10(x, b):
    return  b[0] * x + b[1] / x ** 2 + b[2]

def f10p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = 1 / x ** 2
    grad[2] = 1
    return grad

def initf10b(w=100):
    return [2.78, 1.269, w]
#######################################
def f11(x, b):
    return  b[0] * x ** b[1] + b[2]

def f11p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** b[1]
    grad[1] = b[0] * np.log(x) * x ** b[1]
    grad[2] = 1
    return grad

def initf11b(w=100):
    return [3.58, 0.89, w]
#######################################
def f12(x, b):
    return  b[0] * np.e ** (-b[1] / (x + b[2]))

def f12p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (-b[1] / (x + b[2]))
    grad[1] = b[0] * (-1 / (x + b[2]))
    grad[2] = b[0] * (-b[1]) / (x + b[2]) ** 2 * np.e ** (-b[1] / (x + b[2]))
    return grad

def initf12b(w=100):
    return [692.9 * w / 162.5, -122, 83.2]
#######################################
def f13(x, b):
    return  b[0] * x + b[1] * x ** (1 / 2) + b[2]

def f13p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = x ** (1 / 2)
    grad[2] = 1
    return grad

def initf13b(w=100):
    return [2.31, 1.64, w]
#######################################
def f14(x, b):
    return  b[0] * x + b[1] * x / (x - b[2])

def f14p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = x / (x - b[2])
    grad[2] = b[1] * x / (x - b[2]) ** 2
    return grad

def initf14b(w=100):
    return [2.77, w, 0.00626]
########################################
def f15(x, b):
    return  b[0] * x + b[1] + b[2] / x

def f15p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = 1
    grad[2] = 1 / x
    return grad

def initf15b(w=100):
    return [2.77, w, 0.985]
########################################
def f16(x, b):
    return  b[0] * x + b[1]

def f16p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = 1
    grad[2] = 0
    return grad

def initf16b(w=100):
    return [2.77, w, 0]
#######################################
def f17(x, b):
    return  b[0] * np.e ** (b[1] * x) - b[2]

def f17p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x)
    grad[1] = b[0] * x * np.e ** (b[1] * x) 
    grad[2] = -1
    return grad

def initf17b(w=100):
    return [620 * w / 162.5, 0.00424, -461 * w / 162.5]
########################################
def f18(x, b):
    return  b[0] * x ** b[1] * np.e ** (b[2] * x)

def f18p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** b[1] * np.e ** (b[2] * x)
    grad[1] = b[0] * x * np.log(x) ** b[1] * np.e ** (b[2] * x)
    grad[2] = b[0] * x ** (b[1] + 1) * np.e ** (b[2] * x)
    return grad

def initf18b(w=100):
    return [w, 0.00632, 0.0141]
########################################
def f19(x, b):
    return  b[0] * np.e ** (b[1] * x)

def f19p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x)
    grad[1] = b[0] * x * np.e ** (b[1] * x)
    grad[2] = 0
    return grad

def initf19b(w=100):
    return [w, 0.0155, 0]
#########################################
def f20(x, b):
    return  b[0] * x ** 2 + b[1] * x ** (1 / 2) + b[2]

def f20p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** 2
    grad[1] = x ** (1 / 2)
    grad[2] = 1
    return grad

def initf20b(w=100):
    return [0.0645, 8.52, w]
########################################
def f21(x, b):
    return  b[0] * (np.e ** (b[1] * x) + 1)

def f21p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x) + 1
    grad[1] = b[0] * x * np.e ** (b[1] * x)
    grad[2] = 0
    return grad

def initf21b(w=100):
    return [79.9 * w / 162.5, 0.0286, 0]
#########################################
def f22(x, b):
    return  b[0] * x ** (1 / 2) + b[1]

def f22p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** (1 / 2)
    grad[1] = 1
    grad[2] = 0
    return grad

def initf22b(w=100):
    return [11.4, w, 0]
#########################################
def f23(x, b):
    return  b[0] - b[1] / x + b[2] / x ** 2

def f23p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = 1
    grad[1] = -1 / x
    grad[2] = 1 / x ** 2
    return grad

def initf23b(w=100):
    return [w, -97.9, 69.2]
#########################################
def f24(x, b):
    return  b[0] * x ** (b[1] * x)

def f24p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** (b[1] * x)
    grad[1] = b[0] * x ** 2 * np.log(x) * x ** (b[1] * x)
    grad[2] = 0
    return grad

def initf24b(w=100):
    return [w, 0.0058, 0]
#########################################
def f25(x, b):
    return  (b[0] * x + b[1]) ** b[2]

def f25p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x * b[2] * (b[0] * x + b[1]) ** (b[2] - 1)
    grad[1] = b[2] * (b[0] * x + b[1]) ** (b[2] - 1)
    grad[2] = np.log(b[0] * x + b[1]) * (b[0] * x + b[1]) ** b[2]
    return grad

def initf25b(w=100):
    return [290, 6880 * w / 162.5, 5.7]
#########################################
def f26(x, b):
    return  1 / (b[0] * x ** b[1] - b[2])

def f26p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = -x ** b[1] / (b[0] * x ** b[1] - b[2]) ** 2
    grad[1] = -b[0] * np.log(x) * x ** b[1] / (b[0] * x ** b[1] - b[2]) ** 2
    grad[2] = 1 / (b[0] * x ** b[1] - b[2]) ** 2
    return grad

def initf26b(w=100):
    return [0.0231, -0.0164, -0.0168]
########################################
def f27(x, b):
    return  b[0] * np.e ** (b[1] * x ** 2) - b[2]

def f27p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x ** 2)
    grad[1] = b[0] * x ** 2 * np.e ** (b[1] * x ** 2)
    grad[2] = -1
    return grad

def initf27b(w=100):
    return [2445 * w / 162.5, 0.0000935, -2280 * w / 162.5]
########################################
def f28(x, b):
    return  b[0] * np.e ** (b[1] * x ** 2)

def f28p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x ** 2)
    grad[1] = b[0] * x ** 2 * np.e ** (b[1] * x ** 2)
    grad[2] = 0
    return grad

def initf28b(w=100):
    return [w, 0.00129, 1]
########################################
def f29(x, b):
    return  b[0] * np.log(x) + b[1]

def f29p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x)
    grad[1] = 1
    grad[2] = 0
    return grad

def initf29b(w=100):
    return [10.6, w, 0]
########################################
def f30(x, b):
    return  b[0] * np.log(b[1] * np.log(x + b[2]))

def f30p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(b[1] * np.log(x + b[2]))
    grad[1] = b[0] * np.log(x + b[2]) / (b[1] * np.log(x + b[2]))
    grad[2] = b[0] / np.log(x + b[2]) ** 2
    return grad

def initf30b(w=100):
    return [w, 0.893, 21.2]
########################################

In [43]:
funs = [
    (f1, f1p, initf1b),
    (f2, f2p, initf2b),
    (f3, f3p, initf3b),
    (f4, f4p, initf4b),
    (f5, f5p, initf5b),
    (f6, f6p, initf6b),
    (f7, f7p, initf7b),
    (f8, f8p, initf8b),
    (f9, f9p, initf9b),
    (f10, f10p, initf10b),
    (f11, f11p, initf11b),
    (f12, f12p, initf12b),
    (f13, f13p, initf13b),
    (f14, f14p, initf14b),
    (f15, f15p, initf15b),
    (f16, f16p, initf16b),
    (f17, f17p, initf17b),
    (f18, f18p, initf18b),
    (f19, f19p, initf19b),
    (f20, f20p, initf20b),
    (f21, f21p, initf21b),
    (f22, f22p, initf22b),
    (f23, f23p, initf23b),
    (f24, f24p, initf24b),
    (f25, f25p, initf25b),
    (f26, f26p, initf26b),
    (f27, f27p, initf27b),
    (f28, f28p, initf28b),
    (f29, f29p, initf29b),
    (f30, f30p, initf30b),
]

In [44]:
funs

[(<function __main__.f1(x, b)>,
  <function __main__.f1p(x, b)>,
  <function __main__.initf1b(w=100)>),
 (<function __main__.f2(x, b)>,
  <function __main__.f2p(x, b)>,
  <function __main__.initf2b(w=100)>),
 (<function __main__.f3(x, b)>,
  <function __main__.f3p(x, b)>,
  <function __main__.initf3b(w=100)>),
 (<function __main__.f4(x, b)>,
  <function __main__.f4p(x, b)>,
  <function __main__.initf4b(w=100)>),
 (<function __main__.f5(x, b)>,
  <function __main__.f5p(x, b)>,
  <function __main__.initf5b(w=100)>),
 (<function __main__.f6(x, b)>,
  <function __main__.f6p(x, b)>,
  <function __main__.initf6b(w=100)>),
 (<function __main__.f7(x, b)>,
  <function __main__.f7p(x, b)>,
  <function __main__.initf7b(w=100)>),
 (<function __main__.f8(x, b)>,
  <function __main__.f8p(x, b)>,
  <function __main__.initf8b(w=100)>),
 (<function __main__.f9(x, b)>,
  <function __main__.f9p(x, b)>,
  <function __main__.initf9b(w=100)>),
 (<function __main__.f10(x, b)>,
  <function __main__.f10p(x, b)

In [49]:
def model_growth(vals, days, fun, funp, initb, max_iter=50):
    beta = initb(vals[0])
    
    for n in range(max_iter):
        jacobian = np.zeros((len(days), 3))
        for i in range(len(days)):
            row = costd(vals[i], days[i], beta, fun, funp)
            if i == len(days) - 1:
                row = row / 100
            for j in range(3):
                jacobian[i][j] = row[j]
        residuals = np.array([r_adj(vals[i], x, beta, fun) for i, x in enumerate(days)])
        vec = np.dot(np.linalg.inv(np.dot(np.transpose(jacobian), jacobian)), np.transpose(jacobian))
        shift = np.dot(vec, residuals)
        betas = []
        for s in search:
            betas.append([cost(vals, days, beta - shift / 10.0 ** s, fun), beta - shift / 10.0 ** s])
            betas.append([cost(vals, days, beta + shift / 10.0 ** s, fun), beta + shift / 10.0 ** s])
        og_cost = cost(vals, days, beta, fun)
        min_cost = og_cost
        for b in betas:
            if b[0] < min_cost:
                min_cost = b[0]
                beta = b[1]
        if og_cost == min_cost:
            break
    return min_cost, beta, fun(days[-1], beta)

In [50]:
model_growth(data, xs, f18, f18p, initf18b, 1000)

(141.01117189318506,
 array([1.60160692e+02, 4.50355498e-02, 1.66696056e-03]),
 198.67411731792626)

In [58]:
f, fp, initfb = funs[1]
for i in range(3, len(data)):
    print(model_growth(data[:i], xs[:i], f, fp, initfb, 200)[2])

166.6255527594895
173.09984038615195
163.0682268506856
169.32468767507268
169.1396546837466
171.29828050347763
173.32411918326426
174.73969969989523
175.67141941429475
176.34751443955497
177.08720211005036
177.31777435388548
177.5805330419369
177.99294386929628
178.41623998684005
179.0132366405081
179.49414911861996
180.04920174703287
180.53473576086415
181.0235116014057
181.4238924412544
181.99379850946139
182.42234711705083
182.89077499421614
183.44142782335663
183.91679910465928
184.45358835920095
184.9520560737914
185.2797179570031
185.69295938197448
186.08201771179824
